In [36]:
from typing import List
import trendline
import crypto_data
import equities_data
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# I can add that the signal is a maximum of a fraction of atr away from the breakout line

In [37]:
def combine_market_data(crypto_df=None, equities_df=None):

    dfs_to_concat = []
    
    if crypto_df is not None:
        # Ensure the crypto data has the correct structure
        if not isinstance(crypto_df.columns, pd.MultiIndex):
            raise ValueError("Crypto DataFrame must have MultiIndex columns")
        dfs_to_concat.append(crypto_df)
    
    if equities_df is not None:
        # Ensure the equities data has the correct structure
        if not isinstance(equities_df.columns, pd.MultiIndex):
            raise ValueError("Equities DataFrame must have MultiIndex columns")
        dfs_to_concat.append(equities_df)
    
    if not dfs_to_concat:
        raise ValueError("At least one DataFrame must be provided")
    
    # Concatenate along axis 1 (columns) to preserve the MultiIndex structure
    combined_df = pd.concat(dfs_to_concat, axis=1)
    
    # Sort the columns for better organization
    combined_df = combined_df.sort_index(axis=1)
    
    return combined_df

In [38]:
def trendline_breakout_hl(data: pd.DataFrame, lookback: int, ema_period: int = 21):
    """
    Calculate trendline breakouts with specific pattern conditions:
    Long: price > EMA21 + negative resistance slope
    Short: price < EMA21 + positive resistance slope
    """
    # Calculate EMA
    data['EMA21'] = data['Close'].ewm(span=ema_period, adjust=False).mean()
    
    # Initialize arrays
    n = len(data)
    s_tl = np.full(n, np.nan)
    r_tl = np.full(n, np.nan)
    signals = np.zeros(n)
    r_slopes = np.full(n, np.nan)
    s_slopes = np.full(n, np.nan)
    
    for i in range(lookback, n):
        try:
            # Get window data
            window_close = data.iloc[i - lookback:i]['Close'].to_numpy()
            window_high = data.iloc[i - lookback:i]['High'].to_numpy()
            window_low = data.iloc[i - lookback:i]['Low'].to_numpy()
            
            # Calculate trendlines
            s_coefs, r_coefs = trendline.fit_trendlines_high_low(
                window_high, window_low, window_close
            )
            
            # Store slopes
            s_slopes[i] = s_coefs[0]
            r_slopes[i] = r_coefs[0]
            
            # Project trendlines to current bar
            s_val = s_coefs[1] + lookback * s_coefs[0]
            r_val = r_coefs[1] + lookback * r_coefs[0]
            
            s_tl[i] = s_val
            r_tl[i] = r_val
            
            # Get current price and EMA
            current_close = data.iloc[i]['Close']
            current_ema = data.iloc[i]['EMA21']

            # Average price condition
            window_average = window_close.mean()
            
            # Long condition: price > EMA + negative resistance slope
            if (window_average > current_ema) and (r_coefs[0] < 0) and (current_close > r_val):
                signals[i] = 1.0
                
            # Short condition: price < EMA + positive resistance slope    
            elif (window_average < current_ema) and (r_coefs[0] > 0) and (current_close < s_val):
                signals[i] = -1.0
                
            else:
                signals[i] = 0
                
        except Exception as e:
            signals[i] = signals[i - 1]
            continue
    
    return s_tl, r_tl, signals, data['EMA21'].to_numpy(), r_slopes, s_slopes

# Usage
# s_tl, r_tl, signals, ema, r_slopes, s_slopes = trendline_breakout_hl(data, lookback=14)

In [39]:
# Main loop test

long_tickers = []
short_tickers = []

crypto_data = crypto_data.download_crypto_ohlc_data(periods=30)  
equities_data = equities_data.download_equities_ohlc_data(period="1mo")  

# Combine the data
combined_data = combine_market_data(
    crypto_df=crypto_data,
    equities_df=equities_data
)

print(f"Analyzing {len(combined_data.columns.levels[0])} symbols")


for ticker in combined_data.columns.levels[0]:

    ticker_data = combined_data[ticker].dropna(subset=['Close'])

    print(f"Analyzing {ticker}...")

    s_tl, r_tl, signals, ema, r_slopes, s_slopes = trendline_breakout_hl(ticker_data, 7)
    
    try:
        if signals[-1] == 1:
            long_tickers.append(ticker)
        elif signals[-1] == -1:
            short_tickers.append(ticker)
    except Exception as e:
        print(f'Error {e}')

def categorize_tickers(long_tickers, short_tickers):
    # Initialize categorized lists
    categorized = {
        'crypto': {
            'long': [],
            'short': []
        },
        'equities': {
            'long': [],
            'short': []
        }
    }
    
    # Categorize long tickers
    for ticker in long_tickers:
        if '/USDT:USDT' in ticker:
            categorized['crypto']['long'].append(ticker)
        else:
            categorized['equities']['long'].append(ticker)
    
    # Categorize short tickers
    for ticker in short_tickers:
        if '/USDT:USDT' in ticker:
            categorized['crypto']['short'].append(ticker)
        else:
            categorized['equities']['short'].append(ticker)
    
    return categorized

def print_categorized_results(categorized_tickers):
    """
    Prints the categorized results in a formatted way.
    
    Parameters:
    -----------
    categorized_tickers : dict
        Dictionary containing categorized lists of tickers
    """
    print("\nEquities Results:")
    print("-" * 20)
    print(f"Long: {sorted(categorized_tickers['equities']['long'])}")
    print(f"Short: {sorted(categorized_tickers['equities']['short'])}")
    
    print("\nCrypto Results:")
    print("-" * 20)
    print(f"Long: {sorted(categorized_tickers['crypto']['long'])}")
    print(f"Short: {sorted(categorized_tickers['crypto']['short'])}")

# Categorize the tickers
categorized_results = categorize_tickers(long_tickers, short_tickers)

# Print the results
print_categorized_results(categorized_results)

Initializing data download for: bybit, Timeframe: 1d, Periods: 30...

Data for 10000000AIDOGE/USDT:USDT:
  Start: 2025-01-26 00:00:00+00:00
  End: 2025-02-24 00:00:00+00:00
Last row: 
Timestamp    2025-02-24 00:00:00+00:00
Open                          0.001139
High                          0.001152
Low                           0.001008
Close                         0.001037
Volume                     539127700.0
Name: 29, dtype: object

  Expected Interval: 1d, Actual Interval: 1 days 00:00:00
Finished downloading data for 502 symbols in 28.39 seconds.


[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")')
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=1mo)')


Data downloaded successfully.
Analyzing 1005 symbols
Analyzing 10000000AIDOGE/USDT:USDT...
Analyzing 1000000BABYDOGE/USDT:USDT...
Analyzing 1000000CHEEMS/USDT:USDT...
Analyzing 1000000MOG/USDT:USDT...
Analyzing 1000000PEIPEI/USDT:USDT...
Analyzing 10000COQ/USDT:USDT...
Analyzing 10000ELON/USDT:USDT...
Analyzing 10000LADYS/USDT:USDT...
Analyzing 10000QUBIC/USDT:USDT...
Analyzing 10000SATS/USDT:USDT...
Analyzing 10000WEN/USDT:USDT...
Analyzing 10000WHY/USDT:USDT...
Analyzing 1000APU/USDT:USDT...
Analyzing 1000BONK/USDT:USDT...
Analyzing 1000BTT/USDT:USDT...
Analyzing 1000CAT/USDT:USDT...
Analyzing 1000CATS/USDT:USDT...
Analyzing 1000FLOKI/USDT:USDT...
Analyzing 1000LUNC/USDT:USDT...
Analyzing 1000MUMU/USDT:USDT...
Analyzing 1000NEIROCTO/USDT:USDT...
Analyzing 1000PEPE/USDT:USDT...
Analyzing 1000RATS/USDT:USDT...
Analyzing 1000TOSHI/USDT:USDT...
Analyzing 1000TURBO/USDT:USDT...
Analyzing 1000X/USDT:USDT...
Analyzing 1000XEC/USDT:USDT...
Analyzing 1INCH/USDT:USDT...
Analyzing A...
Analyzin

In [40]:
def plot_ohlcv(data: pd.DataFrame, lookback: int = 7):
    """
    Creates a single plot with OHLCV data and technical indicators.
    """
    # Reset index to get numerical indices for plotting
    data = data.reset_index(drop=True)
    close_prices = data['Close']

    # Calculate trendlines
    s_tl, r_tl, signals, ema, r_slopes, s_slopes = trendline_breakout_hl(data, lookback)
    
    # Create the main figure
    fig = go.Figure()
    
    # Add candlestick chart
    fig.add_trace(
        go.Candlestick(
            x=data.index,
            open=data['Open'],
            high=data['High'],
            low=data['Low'],
            close=data['Close'],
            increasing_line_color='#26A69A',  # Green candles
            decreasing_line_color='#EF5350',  # Red candles
            name='Price'
        )
    )
    
    # Add support trendline
    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=s_tl,
            mode='lines',
            name='Support',
            line=dict(color='white', width=1, dash='dot'),
            showlegend=True
        )
    )
    
    # Add resistance trendline
    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=r_tl,
            mode='lines',
            name='Resistance',
            line=dict(color='white', width=1, dash='dot'),
            showlegend=True
        )
    )
    
    # Add EMA
    fig.add_trace(
        go.Scatter(
            x=data.index,
            y=ema,
            mode='lines',
            name='EMA',
            showlegend=True
        )
    )
    
    # Find long and short signals directly
    long_mask = signals == 1
    short_mask = signals == -1
    
    # Add markers for long signals (up arrows)
    if long_mask.any():
        fig.add_trace(
            go.Scatter(
                x=data.index[long_mask],
                y=close_prices[long_mask],
                mode='markers',
                name='Long Signals',
                marker=dict(
                    size=12,
                    symbol='triangle-up',
                    color='lime',
                    line=dict(color='black', width=2)
                ),
                showlegend=True
            )
        )
    
    # Add markers for short signals (down arrows)
    if short_mask.any():
        fig.add_trace(
            go.Scatter(
                x=data.index[short_mask],
                y=close_prices[short_mask],
                mode='markers',
                name='Short Signals',
                marker=dict(
                    size=12,
                    symbol='triangle-down',
                    color='red',
                    line=dict(color='black', width=2)
                ),
                showlegend=True
            )
        )
    
    # Update layout
    fig.update_layout(
        title='Charlotte',
        xaxis_rangeslider_visible=False,
        height=800,
        yaxis=dict(
            gridcolor='rgba(128,128,128,0.1)',
            zerolinecolor='rgba(128,128,128,0.1)',
        ),
        xaxis=dict(
            gridcolor='rgba(128,128,128,0.1)',
            zerolinecolor='rgba(128,128,128,0.1)',
        ),
        showlegend=True,
        legend=dict(
            bgcolor='rgba(0,0,0,0.5)',
            bordercolor='rgba(255,255,255,0.2)'
        )
    )
    
    return fig

def plot_categorized_results(data: pd.DataFrame, categorized_results: dict):
    """
    Creates plots for categorized crypto and equity signals.
    """
    plots = {}
    
    # Plot crypto signals
    crypto_tickers = categorized_results['crypto']['long'] + categorized_results['crypto']['short']
    if crypto_tickers:
        crypto_fig = make_subplots(
            rows=len(crypto_tickers),
            cols=1,
            subplot_titles=[f"{ticker.split('/')[0]}" for ticker in crypto_tickers],
            vertical_spacing=0.05
        )
        
        for idx, ticker in enumerate(crypto_tickers, 1):
            try:
                ticker_data = data[ticker].copy()  # Make a copy to avoid modifying original
                individual_fig = plot_ohlcv(ticker_data)
                
                for trace in individual_fig.data:
                    crypto_fig.add_trace(trace, row=idx, col=1)
            except Exception as e:
                print(f"Error plotting {ticker}: {str(e)}")
                continue
        
        crypto_fig.update_layout(
            title='Crypto Signals',
            height=400 * len(crypto_tickers),
            showlegend=True,
            legend=dict(
                bgcolor='rgba(0,0,0,0.5)',
                bordercolor='rgba(255,255,255,0.2)'
            )
        )
        
        crypto_fig.update_xaxes(rangeslider_visible=False)
        plots['crypto'] = crypto_fig
    
    # Plot equity signals
    equity_tickers = categorized_results['equities']['long'] + categorized_results['equities']['short']
    if equity_tickers:
        equity_fig = make_subplots(
            rows=len(equity_tickers),
            cols=1,
            subplot_titles=equity_tickers,
            vertical_spacing=0.05
        )
        
        for idx, ticker in enumerate(equity_tickers, 1):
            try:
                ticker_data = data[ticker].copy()  # Make a copy to avoid modifying original
                individual_fig = plot_ohlcv(ticker_data)
                
                for trace in individual_fig.data:
                    equity_fig.add_trace(trace, row=idx, col=1)
            except Exception as e:
                print(f"Error plotting {ticker}: {str(e)}")
                continue
        
        equity_fig.update_layout(
            title='Equity Signals',
            height=400 * len(equity_tickers),
            showlegend=True,
            legend=dict(
                bgcolor='rgba(0,0,0,0.5)',
                bordercolor='rgba(255,255,255,0.2)'
            )
        )
        
        equity_fig.update_xaxes(rangeslider_visible=False)
        plots['equities'] = equity_fig
    
    return plots

# Usage:
"""
# Categorize results
categorized_results = categorize_tickers(long_tickers, short_tickers)

# Create plots
plots = plot_categorized_results(data, categorized_results)

# Display plots
if 'crypto' in plots:
    plots['crypto'].show()
if 'equities' in plots:
    plots['equities'].show()
"""

# Example usage:
# First categorize the results
categorized_results = categorize_tickers(long_tickers, short_tickers)

# Then create the plots
plots = plot_categorized_results(combined_data, categorized_results)

# Show crypto plots if they exist
if 'crypto' in plots:
    plots['crypto'].show()

# Show equity plots if they exist
if 'equities' in plots:
    plots['equities'].show()

ValueError: Vertical spacing cannot be greater than (1 / (rows - 1)) = 0.005882.
The resulting plot would have 171 rows (rows=171).